## We run the methods of the paper on additional social networks
### In this notebook, we run it on large online social networks

In [1]:
# reload imported files automatically without restarting the kernel
%load_ext autoreload
%autoreload 2

In [2]:
from lib import *
from pathcensus import PathCensus
from pathcensus.nullmodels import UBCM
from pathcensus.inference import Inference
from pathcensus.utils import set_seed
from mandarina.benchmark import timer
import networkx as nx


In [3]:
@timer
def calculate_structural_measures(network_name, g, n_samples_null_model):
    """
    This function calculates the structural measures of a network and 
    its null model average values by fitting a UBCM null model to the network.
    """
    n_total = g.number_of_nodes()  # get total number of nodes
    # remove self-loops and multiedges and get largest connected component
    g = preprocess_graph(g)  ; g
    n_giant = g.number_of_nodes()  # get number of nodes in largest connected component
    degseq = sorted([d for n, d in g.degree()], reverse=True)  # get degree sequence
    dataset = ""
    network_name = network_name.split(".")[0]
    label = "large online"
    
    properties = {
            "idx": 1,
            "dataset": dataset,
            "name": network_name,
            #"graph": [g],  # get graph
            "n_nodes": n_giant,  # get number of nodes in largest connected component
            "frac_total": n_giant
            / n_total,  # get fraction of nodes in largest connected component
            "density": nx.density(g),  # get density
            "dbar": np.mean(degseq),  # get mean degree
            "dcv": np.std(degseq)
            / np.mean(degseq),  # get coefficient of variation of degree
            "dmax": np.max(degseq),  # get maximum degree
        }
    
    model = UBCM(g)  # initialize model
    model.fit()  # fit model
    model.validate()  # validate model
    # compare null model to actual graph using statistics function
    infer = Inference(g, model, statistics)
    data, null = infer.init_comparison(n=n_samples_null_model)
    original_network_values = dict(data)
    original_network_values = {'data_' + k: v for k, v in original_network_values.items()}
    null_model_mean_values = dict(null.mean(numeric_only=True)) 
    null_model_mean_values = {f'null_{n_samples_null_model}_{k}': v for k, v in null_model_mean_values.items()}
    return properties | null_model_mean_values |original_network_values | null_model_mean_values

In [4]:
OUTPUT_CSV_FILE_PATH = RESULT_DIR_PATH / "structural_measures_large_online.csv"

In [5]:
def dataset_size_filter(dataset_path, size):
    """Filter datasets by size, given in bytes."""
    return dataset_path.stat().st_size < size

In [6]:
file_filter = lambda file: file.is_file()
large_online_network_files = list(filter(file_filter, (DATA_DIR_PATH / "online" / "large").glob("**/*")))

filtered_dataset_paths = list(filter(lambda x: dataset_size_filter(x, 50000000), large_online_network_files))

print(len(filtered_dataset_paths))

results = []
for i, file in enumerate(filtered_dataset_paths): # loop over all online large social network files
    f = gml_cleaner(file) # clean gml file
    g = nx.read_gml(f, label="id") # load into networkx
    print("Running calculations for network: ", file.name) 
    # calculate measures from original network and null model
    result = calculate_structural_measures(file.name.split(".")[0] , g, n_samples_null_model=10) 
    #df = pd.DataFrame(result).reset_index()
    #df[df.columns[:-1]].to_csv(OUTPUT_CSV_FILE_PATH, mode='a', header=not OUTPUT_CSV_FILE_PATH.is_file()) # append result row to csv file
    results.append(result) # collect results


33
Running calculations for network:  NYC_restaurant_tips.gml
Function calculate_structural_measures with args (('NYC_restaurant_tips', <networkx.classes.multigraph.MultiGraph object at 0x7f626ad56110>), {'n_samples_null_model': 10}) took: 14.3793 seconds.
Running calculations for network:  NYC_restaurant_checkin.gml
Function calculate_structural_measures with args (('NYC_restaurant_checkin', <networkx.classes.multigraph.MultiGraph object at 0x7f6268afcd30>), {'n_samples_null_model': 10}) took: 3.0401 seconds.
Running calculations for network:  douban.gml


In [ ]:
df = pd.concat([pd.DataFrame(result) for result in results]).reset_index() # combine all data to dataframe for analysis
df = df[df.columns[:-1]]

In [ ]:
df

,_,idx,dataset,name,n_nodes,frac_total,density,dbar,dcv,dmax,...,null_10_sim_e,null_10_comp_g,null_10_comp,null_10_comp_e,data_sim_g,data_sim,data_sim_e,data_comp_g,data_comp,data_comp_e
0,0,1,,NYC_restaurant_tips,5372,0.838066,0.000614,3.295607,1.569503,196,...,0.003222,0.004665,0.001381,0.002550,0.000000,0.000000,0.000000,0.014416,0.003092,0.006515
1,0,1,,NYC_restaurant_checkin,4906,0.993922,0.001118,5.485936,1.044863,88,...,0.003277,0.004657,0.002076,0.003273,0.000000,0.000000,0.000000,0.011630,0.005808,0.008769
2,0,1,,gplus_106228758905254036967,4839,0.999380,0.022479,108.752222,1.153716,1233,...,0.090986,0.065140,0.036865,0.057262,0.301261,0.213295,0.310858,0.060981,0.030823,0.049776
3,0,1,,gplus_112573107772208475213,3794,0.998684,0.008368,31.738535,1.780662,548,...,0.085733,0.052095,0.020929,0.044165,0.174692,0.083500,0.169532,0.060478,0.023330,0.050768
4,0,1,,gplus_115455024457484679647,3464,0.992550,0.018095,62.663395,1.815547,919,...,0.182141,0.049314,0.022843,0.047710,0.284835,0.112724,0.259228,0.039766,0.023916,0.045025
5,0,1,,gplus_106186407539128840569,4289,1.000000,0.026793,114.888785,1.398220,1165,...,0.164846,0.073646,0.037245,0.067648,0.367212,0.188241,0.332750,0.044179,0.022321,0.042289
6,0,1,,facebook_combined,4039,1.000000,0.010820,43.691013,1.199654,1045,...,0.044034,0.038238,0.020748,0.032339,0.519174,0.312620,0.513878,0.019899,0.014564,0.022281
7,0,1,,gplus_104905626100400792399,4472,0.997769,0.026578,118.830501,1.288651,1869,...,0.124919,0.064524,0.037284,0.058846,0.264620,0.172543,0.263097,0.054063,0.032271,0.050277
8,0,1,,gplus_115273860520983542999,4520,1.000000,0.026075,117.833628,1.504510,1886,...,0.162169,0.056255,0.030533,0.053233,0.212616,0.111475,0.203295,0.069743,0.037160,0.063614
9,0,1,,gplus_114147483140782280818,4227,0.999527,0.048591,205.343743,1.122595,1796,...,0.172821,0.071148,0.047754,0.067223,0.232945,0.155755,0.219303,0.084954,0.056936,0.080021


In [ ]:
df.to_csv(RESULT_DIR_PATH / "structural_measures_large_online.csv", index=False)